# 1º Parte

In [1]:
!pip install requests python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
#Eu criei variaveis de ambiente para passar para o GitHub sem problemas

from dotenv import load_dotenv
import os

load_dotenv()
tlt_key = os.getenv("TLT_API_KEY")
oai_key = os.getenv("OAI_API_KEY")

In [19]:
import requests
from docx import Document


subscription_key = tlt_key
endpoint = "https://api.cognitive.microsofttranslator.com"
location = "brazilsouth"
target_language = 'pt-br'


def translator_text(text, target_language):
  path = '/translate'
  constructed_url = endpoint + path
  headers = {
      "Ocp-Apim-Subscription-Key": subscription_key,
      "Ocp-Apim-Subscription-Region": location,
      "Content-type": "application/json",
      "X-ClientTraceId": str(os.urandom(16))
  }

  body = [{
      "text": text
  }]

  params = {
    'api-version': '3.0',
    'from': 'en',
    'to': target_language
  }

  request = requests.post(constructed_url, params=params, headers=headers, json=body)

  response = request.json()

  return response[0]['translations'][0]['text']
  # return response



In [20]:
texto = "Soft skills refer to a combination of interpersonal skills, communication abilities, and emotional intelligence that enable individuals to work effectively with others. "

translator_text(texto, target_language)

'As soft skills referem-se a uma combinação de habilidades interpessoais, habilidades de comunicação e inteligência emocional que permitem que os indivíduos trabalhem efetivamente com os outros. '

In [25]:
def translator_document(path):
  document = Document(path)
  full_text = []
  for paragrafh in document.paragraphs:
    translated_text = translator_text(paragrafh.text, target_language)
    full_text.append(translated_text)

  translated_doc = Document()
  for line in full_text:
    translated_doc.add_paragraph(line)

  path_translated = path.replace('.docx', f'_{target_language}.docx')
  translated_doc.save(path_translated)


In [26]:
input_file = "/content/Soft_skills.docx"

translator_document(input_file)

# 2º Parte

In [27]:
!pip install requests beautifulsoup4 openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00


In [32]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
      text = soup.get_text(separator=" ")
      #Limpar Texto
      lines = (line.strip() for line in text.splitlines())
      parts = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(part for part in parts if part)
    else:
      print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
      return None

    return text

extract_text_from_url("https://dev.to/opire/11-practical-ways-to-bring-side-income-as-a-developer-5apm")

'11 Practical Ways to Bring Side Income as a Developer 💰 - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by\nSearch\nAlgolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nAnmol Baranwal\nfor\nOpire\nPosted on\nNov 20\n11 Practical Ways to Bring Side Income as a Developer 💰\n# programming\n# beginners\n# opensource\n# productivity\nIt’s always smart not to rely on just one source of income.\nBringing in side income might seem tough at first, but for developers, it’s surprisingly easier than you think.\nThis post briefly outlines 21 practical ways to build some side revenue as a developer.\nThis guide will help you build a sustainable income.\n1. Solve open source bounties.\nThis is one of the most underrated ways and is often neglected.\nI

In [ ]:
from langchain_openai.chat_models import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint="https://oai-dio-bootcamp-dev-eastus-001-caio.openai.azure.com/",
    api_key= oai_key,
    api_version= "2024-02-15-preview",
    deployment_name="gpt-4o-mini",
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system","Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  print(response.content)
  return(response.content)

In [43]:
translate_article("Let's see if the deployment was succeded.", "portugues")

Vamos ver se o deployment foi bem-sucedido.


'Vamos ver se o deployment foi bem-sucedido.'

In [44]:
url = "https://dev.to/opire/11-practical-ways-to-bring-side-income-as-a-developer-5apm"
text = extract_text_from_url(url)
article = translate_article(text, "portugues")

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}